In [1]:
import pandas as pd
customer_profiles_table=pd.read_pickle("../data/raw/customer.pkl")

In [2]:
customer_profiles_table.head()

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals,nb_terminals
0,0,54.881350,71.518937,62.262521,31.131260,2.179533,"[29, 87, 144, 241, 330, 858, 996, 1028, 1067, ...",78
1,1,42.365480,64.589411,46.570785,23.285393,3.567092,"[5, 160, 242, 378, 431, 475, 571, 762, 876, 93...",85
2,2,96.366276,38.344152,80.213879,40.106939,2.115580,"[316, 406, 447, 523, 968, 1200, 1318, 1365, 16...",70
3,3,56.804456,92.559664,11.748426,5.874213,0.348517,"[65, 94, 113, 364, 401, 433, 485, 651, 672, 77...",70
4,4,2.021840,83.261985,78.924891,39.462446,3.480049,"[372, 614, 774, 1362, 1446, 1564, 1637, 1939, ...",65


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField

In [4]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("data_store")
         .getOrCreate())

25/02/13 21:09:14 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
25/02/13 21:09:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c90e74e2-2393-4d1e-9945-6d184e7b3ca3;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [5]:

# Select specific columns
customer_sdf =customer_profiles_table[["CUSTOMER_ID", "x_customer_id", "y_customer_id"]]

In [6]:
customer_sdf=spark.createDataFrame(customer_sdf)

In [7]:
customer_sdf=customer_sdf.withColumnRenamed("CUSTOMER_ID","customer_id")
customer_sdf=customer_sdf.withColumnRenamed("x_customer_id","x_location")
customer_sdf=customer_sdf.withColumnRenamed("y_customer_id","y_location")

In [8]:
from pyspark.sql.functions import current_timestamp
customer_sdf=customer_sdf.withColumn("row_created_timestamp", current_timestamp())
customer_sdf=customer_sdf.withColumn("row_updated_timestamp", current_timestamp())

In [9]:
customer_sdf.createOrReplaceTempView("customer_sdf")

25/02/13 21:10:38 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [11]:
spark.sql("""
CREATE TABLE IF NOT EXISTS payment.customer (
    customer_id INT,
    x_location FLOAT,
    y_location FLOAT,
    row_created_timestamp TIMESTAMP,
    row_updated_timestamp TIMESTAMP
)
USING iceberg
""").show(truncate=False)

++
||
++
++



In [12]:
spark.sql("""
MERGE INTO payment.customer AS target
USING customer_sdf AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
  UPDATE SET
    target.x_location = source.x_location,
    target.y_location = source.y_location,
    target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    customer_id,
    x_location,
    y_location,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.customer_id,
    source.x_location,
    source.y_location,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""")

DataFrame[]

In [13]:
customer_sdf.show()

+-----------+------------------+------------------+---------------------+---------------------+
|customer_id|        x_location|        y_location|row_created_timestamp|row_updated_timestamp|
+-----------+------------------+------------------+---------------------+---------------------+
|          0| 54.88135039273247| 71.51893663724195| 2025-02-13 21:14:...| 2025-02-13 21:14:...|
|          1| 42.36547993389047| 64.58941130666561| 2025-02-13 21:14:...| 2025-02-13 21:14:...|
|          2| 96.36627605010293| 38.34415188257777| 2025-02-13 21:14:...| 2025-02-13 21:14:...|
|          3| 56.80445610939323|  92.5596638292661| 2025-02-13 21:14:...| 2025-02-13 21:14:...|
|          4| 2.021839744032572|  83.2619845547938| 2025-02-13 21:14:...| 2025-02-13 21:14:...|
|          5|  97.8618342232764| 79.91585642167236| 2025-02-13 21:14:...| 2025-02-13 21:14:...|
|          6|11.827442586893323| 63.99210213275238| 2025-02-13 21:14:...| 2025-02-13 21:14:...|
|          7|52.184832175007166| 41.4661